<a href="https://colab.research.google.com/github/youneslebboukh/text-summarization/blob/main/txt_summ_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"youneslbk","key":"ff30355568a25aef64e5c064a14e36fc"}'}

In [3]:
 !mkdir -p ~/.kaggle
 !cp kaggle.json ~/.kaggle/

In [6]:
  !chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d allen-institute-for-ai/CORD-19-research-challenge

CORD-19-research-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)


In [8]:
!pip install zip_files

In [ ]:
from zipfile import ZipFile
file_name="CORD-19-research-challenge.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.6/zipfile.py", line 1579, in _extract_member
    shutil.copyfileobj(source, target)
  File "/usr/lib/python3.6/shutil.py", line 79, in copyfileobj
    buf = fsrc.read(length)
  File "/usr/lib/python3.6/zipfile.py", line 872, in read
    data = self._read1(n)
  File "/usr/lib/python3.6/zipfile.py", line 948, in _read1
    data = self._decompressor.decompress(data, n)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-b029c635ba4c>", line 4, in <module>
    zip.extractall()
  File "/usr/lib/python3.6/zipfile.py", line 1524, in extractall
    self._extract_member(zipinfo, path, pwd)
  File "/usr/lib/python3.6/zipfile.py", line 1579, in _extract_member
    shutil.copyfileobj(sour

In [1]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
metadata = pd.read_csv("/content/metadata.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
metadata.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


In [4]:
import json
import glob

In [5]:
p="/content/document_parses"
all_json = glob.glob(f'{p}/**/*.json', recursive=True)
len(all_json)

240409

In [6]:
full_body_text=[]
#for i in range(5) :
  #with open(all_json[i]) as file :
with open(all_json[0]) as file :
    content = json.load(file)
    abstract = []
    body_text = []
    # Body text
    for entry in content['body_text']:
      body_text.append(entry['text'])
    full_body_text.append(body_text)
    body_text = '\n'.join(body_text)
    # Abstract
    #for entry in content['abstract']:
     # abstract.append(entry['text'])
    #abstract = '\n'.join(abstract)

In [7]:
for i in full_body_text :
  print(i)

['Low level laser therapy (LLLT) is also known as cold laser therapy or photobiomodulation therapy. LLLT utilizes visible light and infrared laser beams in the range of 450–1000 nm. Single wavelength or monochromatic light is emitted from a low-intensity laser diode (<500 mW). The light source is placed in contact with the skin, allowing the photon energy to penetrate tissue, where it interacts with various intracellular biomolecules to restore normal cell function and enhance the body’s healing processes [1]. This contrasts with the thermal effects produced by the high-power lasers that are used in cosmetic and surgical procedures to destroy tissue [1], as mentioned in the PubMed Medical Subject Heading (MeSH) subheading for LLLT.', 'LLLT effects are not due to heat but rather to a photochemical reaction that occurs when a photoacceptor molecule within the cell absorbs a photon of light, becomes activated, and changes the cell’s membrane permeability and metabolism. Presently, cytochr

In [8]:
def preprocessing(text):
    # remove mail
    text = re.sub(r'[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}', 'MAIL', text)
    # remove doi
    text = re.sub(r'https\:\/\/doi\.org[^\s]+', 'DOI', text)
    # remove https
    text = re.sub(r'(\()?\s?http(s)?\:\/\/[^\)]+(\))?', '\g<1>LINK\g<3>', text)
    # remove single characters repeated at least 3 times for spacing error (e.g. s u m m a r y)
    text = re.sub(r'(\w\s+){3,}', ' ', text)
    # replace tags (e.g. [3] [4] [5]) with whitespace
    text = re.sub(r'(\[\d+\]\,?\s?){3,}(\.|\,)?', ' \g<2>', text)
    # replace tags (e.g. [3, 4, 5]) with whitespace
    text = re.sub(r'\[[\d\,\s]+\]', ' ',text)
     # replace tags (e.g. (NUM1) repeated at least 3 times with whitespace
    text = re.sub(r'(\(\d+\)\s){3,}', ' ',text)
    # replace '1.3' with '1,3' (we need it for split later)
    text = re.sub(r'(\d+)\.(\d+)', '\g<1>,\g<2>', text)
    # remove all full stops as abbreviations (e.g. i.e. cit. and so on)
    text = re.sub(r'\.(\s)?([^A-Z\s])', ' \g<1>\g<2>', text)
    # correctly spacing the tokens
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r'\.{2,}', '.', text)
    text=re.sub(r"[<>()(,)|&©ø\[\]\'\";?~*!]", ' ', text) #remove <>()|&©ø"',;?~*!
    text=re.sub("(\\t)", ' ', text) #remove escape charecters
    text=re.sub("(\\r)", ' ', text) 
    text=re.sub("(\\n)", ' ', text)
    text= re.sub("(\s+)",' ',text) #remove multiple space
    # return lowercase text
    return text.lower()

In [9]:
from nltk.tokenize import sent_tokenize
full_body_text2=[]

In [10]:
for i in full_body_text :
  txt = preprocessing(i[0])
  t=sent_tokenize(txt)
  full_body_text2.append(t)

In [11]:
for s in full_body_text2 :
  print(s)

['low level laser therapy lllt is also known as cold laser therapy or photobiomodulation therapy.', 'lllt utilizes visible light and infrared laser beams in the range of 450–1000 nm.', 'single wavelength or monochromatic light is emitted from a low-intensity laser diode 500 mw .', 'the light source is placed in contact with the skin allowing the photon energy to penetrate tissue where it interacts with various intracellular biomolecules to restore normal cell function and enhance the body’s healing processes .', 'this contrasts with the thermal effects produced by the high-power lasers that are used in cosmetic and surgical procedures to destroy tissue as mentioned in the pubmed medical subject heading mesh subheading for lllt.']


In [12]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [14]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [15]:
clean_body_text = []
for r in full_body_text2 :
  c_body_text = []
  for rr in r :
    c_body_text.append(remove_stopwords(rr.split()))
  clean_body_text.append(c_body_text)


for s in clean_body_text :
  print(s)

['low level laser therapy lllt also known cold laser therapy photobiomodulation therapy.', 'lllt utilizes visible light infrared laser beams range 450–1000 nm.', 'single wavelength monochromatic light emitted low-intensity laser diode 500 mw .', 'light source placed contact skin allowing photon energy penetrate tissue interacts various intracellular biomolecules restore normal cell function enhance body’s healing processes .', 'contrasts thermal effects produced high-power lasers used cosmetic surgical procedures destroy tissue mentioned pubmed medical subject heading mesh subheading lllt.']


In [16]:
from nltk.cluster.util import cosine_distance
import networkx as nx

In [18]:
def sentence_similarity(sent1,sent2,stopwords=None):
  if stopwords is None :
    stopwords = []
  sent1=[w.lower() for w in sent1]
  sent2=[w.lower() for w in sent2]
  all_words=list(set(sent1+sent2))
  vector1=[0]*len(all_words)
  vector2=[0]*len(all_words)
  for w in sent1 :
    if w in stopwords :
      continue
    vector1[all_words.index(w)] += 1
  for w in sent2 :
    if w in stopwords :
      continue
    vector2[all_words.index(w)] += 1
  return 1-cosine_distance(vector1,vector2)

In [24]:
def gen_sim_matrix(sentences,stop_words):
  similarity_matrix=np.zeros(len(sentences),len(sentences))
  for idx1 in range(len(sentences)) :
    for idx2 in range(len(sentences)) :
      if idx1 == idx2 :
        continue
      similarity_matrix[idx1][idx2]=sentence_similarity(sentences[idx1],sentences[idx2],stop_words)
  return similarity_matrix

In [34]:
def generate_summary(filename,top_n=5) :
  stop_words=stopwords.words('english')
  summarize_text=[]
  s=clean_body_text[0]
  sentences=s_new = " ".join([i for i in clean_body_text[0]])
  sentence_similarity_matrix=gen_sim_matrix(sentences,stop_words)
  sentence_similarity_graph=nx.from_numpy_array(sentence_similarity_matrix)
  scores=nx.pagerank(sentence_similarity_graph)
  ranked_sentence=sorted(((scores[i],s) for i,s in enumerate(sentences)),reverse=True)
  for i in range(top_n):
    summarize_text.append(" ".join(ranked_sentence[i][1]))
  print("summary \n",". ".join(summarize_text))


In [33]:
s_new = " ".join([i for i in clean_body_text[0]])

In [35]:
generate_summary(s_new)

TypeError: ignored